In [ ]:
#Server prg
import socket,math,datetime,random,sys,os
import numpy as np
import pandas as pd
from scipy import constants
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
#import plotly.express as px
#import altair as alt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def is_float(a_string):
    try:
        float(a_string)
        return True
    except ValueError:
        return False
try:
    s=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((socket.gethostname() ,1234))  
    print(socket.gethostname())
    s.listen(5)
    while True:
        clt,adr=s.accept()
        msg=clt.recv(64000)
        xcode=msg.decode("utf-8")
        #print(xcode)
        if(xcode=="exit()" or xcode=="quit()"):
            print("output-0: ",xcode)
            sys.exit("You have stoped") 
            clt.close()
        elif(xcode.endswith(".csv")):
            df = pd.read_csv(xcode)
            print("output-00: ",xcode," loaded")
            clt.send("pandas dataframe object with 'df' loaded.".encode())
        elif(xcode.startswith("chart")):
            #if os.path.exists("plot.jpg"):
            #    os.remove("plot.jpg")
            #else:
            #    print("The file does not exist")
               
            plt.savefig("plot.jpg") # format='svg' dpi=300)   
            plt.close()
            with open('plot.jpg', 'rb') as file:
                image_data = file.read()
                clt.sendall(image_data)
        else:
            exec(f"out= "+xcode) #, globlsparam,  localsparam)
            output=str(out)
            
            if(output.isnumeric()):
                clt.send(bytes(out))
                print("output-1: ",out)
            elif (is_float(output)):
                clt.send(output.encode())
                print("output-1: ",output)
            else:
                clt.send(output.encode())
                print("output-2: ",output)
except Exception as err:
    print(f"Unexpected {err=}")
    error=str('Runtime Error: ') + str(err)
    clt.send(error.encode())
    clt.close()
    raise    

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Generate some data
x = np.linspace(0, 2 * np.pi, 100)
y = np.sin(x)

# Create a line chart
plt.plot(x, y)

# Save the plot to a file
plt.savefig("plot.png")

In [ ]:
pip install altair

In [ ]:
import seaborn as sns
print(sns.__version__)

In [ ]:
pip list

In [ ]:
import altair as alt
import pandas as pd
  
# Making a Pandas DataFrame
score_data = pd.DataFrame({
    'Website': ['StackOverflow', 'FreeCodeCamp',
                'GeeksForGeeks', 'MDN', 'CodeAcademy'],
    'Score': [65, 50, 99, 75, 33]
})
  
# Making the Simple Bar Chart
alt.Chart(score_data).mark_bar().encode(
    # Mapping the Website column to x-axis
    x='Website',
    # Mapping the Score column to y-axis
    y='Score'
)

In [ ]:
code="a=10, b=20, c=a+b"
x=code.split(",")
#print(x)
for i in x: 
    print(i) 

In [ ]:
#Server prg
import socket,math,datetime,random,sys,os
import numpy as np
import pandas as pd
from scipy import constants
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
#import altair as alt

def is_float(a_string):
    try:
        float(a_string)
        return True
    except ValueError:
        return False
try:
    s=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((socket.gethostname() ,1234))  
    print(socket.gethostname())
    s.listen(5)
    while True:
        clt,adr=s.accept()
        msg=clt.recv(24576)
        code=msg.decode("utf-8")
        x = code.split("\n")
        for i in x:
            xcode=i
            if(xcode=="exit()" or xcode=="quit()"):
                print("output-0: ",xcode)
                sys.exit("You have stoped") 
                clt.close()
            elif(xcode.endswith(".csv")):
                df = pd.read_csv(xcode)
                print("output-00: ",xcode," loaded")
                clt.send("pandas dataframe object with 'df' loaded.".encode())
            elif(xcode.startswith("chart")):
                plt.savefig("plot.jpg")
            
                with open('plot.jpg', 'rb') as file:
                    image_data = file.read()
                clt.sendall(image_data)    
            else:
                exec(f"out= "+xcode) #, globlsparam,  localsparam)
                output=str(out)
            
                if(output.isnumeric()):
                    clt.send(bytes(out))
                    print("output-1: ",out)
                elif (is_float(output)):
                    clt.send(output.encode())
                    print("output-1: ",output)
                else:
                    clt.send(output.encode())
                    print("output-2: ",output)
except Exception as err:
    print(f"Unexpected {err=}")
    error=str('Runtime Error: ') + str(err)
    clt.send(error.encode())
    clt.close()
    raise    

In [ ]:
import seaborn as sns
import plotly.graph_objs as go

# Load data
tips = sns.load_dataset("tips")

# Create plot
sns.scatterplot(x="total_bill", y="tip", hue="sex", data=tips)

# Convert plot to Plotly object
plotly_fig = go.Figure(data=go.Scatter(x=tips["total_bill"], y=tips["tip"], mode="markers", marker=dict(color=tips["sex"])))

# Export Plotly object to HTML file
plotly_fig.write_html("seaborn_plot.html")


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('data.csv')
sns.set_theme()
ai_cor=df.corr()
sns.heatmap(ai_cor,annot=True,cmap='coolwarm',linewidths=1)

#ds = sns.load_dataset(df).pivot(index=str("areas"), columns="rooms", values="price")
#sns.heatmap(ds)

In [ ]:
numbers = [3.0, 3.0, 3.0, 2.0, 4.0, 4.0, 3.0, 3.0, 3.0, 3.0, 4.0, 3.0, 3.0, 5.0, 3.0, 4.0, 2.0, 3.0, 4.0, 4.0, 3.0, 2.0, 3.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 2.0, 1.0, 4.0, 3.0, 4.0, 3.0, 3.0, 4.0, 4.0, 2.0, 3.0, 4.0, 3.0, 2.0, 4.0, 3.0]
roomlist=[numbers.count(1.0),numbers.count(2.0),numbers.count(3.0),numbers.count(4.0),numbers.count(5.0)]
print(roomlist)